In [1]:
import requests
import itertools
import logging
from bs4 import BeautifulSoup
import time
import re
from tqdm import tqdm_notebook
import warnings
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

warnings.filterwarnings('ignore')

Задача - получить данные по объявлениям о продаже автомобилей с сайта avito.ru. Интересные нам поля: рейтинг, название авто в объявлении, цена, комплектация и некоторые технические характеристики автомобиля. Парсить будем с помощью Selenium и фреймворка stealth для имитации деятельности живого человека и избежания блокировки ip-адреса. Также будем логировать ключевые моменты во время парсинга.

In [2]:
logging.basicConfig(
    level=logging.INFO,
    filename="parse_log.log",
    format=
    "%(asctime)s - %(levelname)s - %(funcName)s: %(lineno)d - %(message)s"
)

Создадим небольшой словарик, который поможет нам работать с ссылками авито.

In [3]:
brands_dict = {
    'audi': 'ASgBAgICAUTgtg3elyg', 
    'bmw': 'ASgBAgICAUTgtg3klyg', 
    'changan': 'ASgBAgICAUTgtg3wlyg', 
    'chery': 'ASgBAgICAUTgtg30lyg', 
    'chevrolet': 'ASgBAgICAUTgtg32lyg',
    'citroen': 'ASgBAgICAUTgtg36lyg',
    'exeed': 'ASgBAgICAUTgtg24ldEB', 
    'ford': 'ASgBAgICAUTgtg2cmCg', 
    'geely': 'ASgBAgICAUTgtg2gmCg',
    'haval': 'ASgBAgICAUTgtg2umCg', 
    'honda': 'ASgBAgICAUTgtg2ymCg', 
    'hyundai': 'ASgBAgICAUTgtg2imzE', 
    'kia': 'ASgBAgICAUTgtg3KmCg', 
    'land_rover': 'ASgBAgICAUTgtg3QmCg', 
    'lexus': 'ASgBAgICAUTgtg3WmCg', 
    'mazda': 'ASgBAgICAUTgtg3mmCg',
    'mercedes-benz': 'ASgBAgICAUTgtg3omCg', 
    'mitsubishi': 'ASgBAgICAUTgtg3ymCg', 
    'nissan': 'ASgBAgICAUTgtg36mCg', 
    'opel': 'ASgBAgICAUTgtg3', 
    'peugeot': 'ASgBAgICAUTgtg2AmSg', 
    'renault': 'ASgBAgICAUTgtg2MmSg',
    'skoda': 'ASgBAgICAUTgtg2emSg', 
    'toyota': 'ASgBAgICAUTgtg20mSg', 
    'volkswagen': 'ASgBAgICAUTgtg24mSg', 
    'volvo': 'ASgBAgICAUTgtg26mSg', 
    'vaz_lada': 'ASgBAgICAUTgtg3GmSg', 
    'gaz': 'ASgBAgICAUTgtg3KmSg'
}

In [4]:
class AvitoParser:
    ''' 
    Класс для парсинга авто по бренду постранично с сайта avito.ru.
    Каждый бренд имеет в своей ссылке уникальную часть, поэтому словарь с 
    ними нужно подать на вход классу. 
    '''
    def __init__(self, brands_dict: dict):
        ''' 
        Инициализируем переменные, которые нам понадобятся в будущем
        ---------------------------------------------------------------
        :brands_dict: dict - словарь с уникальными частями ссылок по брендам.
        :page_start: int - номер страницы, с которой начнём парсить.
        :page_stop: int - номер страницы, на которой закончим парсить.
        :brand: str - бренд авто.
        :links_archive: list - архив с ссылками на автомобили, 
        которые мы получим при парсинге страниц. 
        '''
        self.brands_dict = brands_dict
        self.page_start = None
        self.page_stop = None
        self.brand = None
        self.links_archive = None

    def __browser_run(self) -> None:
        ''' 
        Встроенный метод для запуска браузера Chrome с помощью 
        библиотеки Selenium в "скрытном" режиме для
        обхода возможных блокировок. Браузер запускается с 
        двумя вкладками и оставляет первую вкладку активной. 
        '''
        try:
            options = webdriver.ChromeOptions()
            options.add_argument("start-maximized")
            options.add_experimental_option("excludeSwitches",
                                            ["enable-automation"])
            options.add_experimental_option('useAutomationExtension', False)
            self.driver = webdriver.Chrome(options=options)
            stealth(
                self.driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win64",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True,
            )
            self.driver.switch_to.new_window('tab')
            self.driver.switch_to.window(self.driver.window_handles[0])
            logging.info('Успешно открылся браузер с двумя вкладками.')
        except Exception as err:
            logging.error(f'Браузер не открылся: {err}.')

    def __get_url(self, page_num: int) -> None:
        ''' 
        Встроенный метод получения необходимой нам страницы по ссылке. 
        --------------------------------------------------------
        :page_num: int - номер страницы, которую будем парсить. 
        '''
        self.driver.get(
            f'https://www.avito.ru/kursk/avtomobili/{self.brand}'
            f'-{self.brands_dict[self.brand]}~mCg?cd=1&p={page_num}'
            f'&radius=3000&searchRadius=3000')
        time.sleep(np.random.randint(6, 8))

    def __get_links(self) -> None:
        ''' 
        Встроенный метод для получения ссылок на 
        машины с нашей текущей страницы. 
        Добавляет ссылки во временный список links_archive. 
        '''
        try:
            block = self.driver.find_element(By.CLASS_NAME,
                                             'items-items-kAJAg')
            links = block.find_elements(By.CSS_SELECTOR,
                                        '[data-marker="item-title"]')
            for link in links:
                self.links_archive.append(link.get_attribute('href'))
            logging.info('Успешно получены ссылки на машины.')
        except Exception as err:
            logging.error(f'Ошибка в получении ссылок на машины: {err}.')

    def __parse_page_to_df(self, page_num: int) -> None:
        ''' 
        Встроенный метод для парсинга информации 
        непосредственно по ссылкам на машины, 
        которые мы получали во временный список.
        ---------------------------------------------------------------------
        :page_num: int - номер страницы, которую парсим. 
        '''
        try:
            self.driver.switch_to.window(self.driver.window_handles[1])
            data = pd.DataFrame()
            for link in self.links_archive:
                logging.info(f'Парсим ссылку {link}')
                self.driver.get(link)
                time.sleep(np.random.randint(11, 15))
                soup = BeautifulSoup(self.driver.page_source, 'lxml')
                rating_badge = self.driver.find_elements(
                    By.CLASS_NAME, "RatingBadge-root-ob0UI")
                if rating_badge:
                    car_rating = soup.find('strong',
                                           class_=[
                                               'styles-module-root-bLKnd',
                                               'styles-module-size_s-AGMw8'
                                           ]).text

                else:
                    car_rating = 'Отсутствует'
                car_name = soup.find(
                    'h1',
                    class_=[
                        'styles-module-root-GKtmM', 'styles-module-root-YczkZ',
                        'styles-module-size_xxxl-MrhiK',
                        'styles-module-size_xxxl-c1c6_',
                        'stylesMarningNormal-module-root-S7NIr',
                        'stylesMarningNormal-module-header-3xl-scwbO'
                    ]).text
                car_price = soup.find(
                    'span', class_="style-price-value-main-TIg6u").text
                car_price = re.sub(r'[^\d*]', '', car_price)
                car_location = soup.find(
                    'span', class_="style-item-address__string-wt61A").text
                car_options = soup.find_all(
                    'ul', class_="style-advanced-params-column-cRfzo")
                options_list = []

                for option in car_options:
                    options_list.extend(
                        re.findall(r'[А-ЯЁ][^А-ЯЁ]*', option.text))

                opt_values = [
                    x for x in options_list if x not in options_list[::2]
                ]
                opt_names = options_list[::2]
                params_block = soup.find_all(
                    'li', class_="params-paramsList__item-_2Y2O")
                cols = ['Название авто', 'Рейтинг', 'Цена', 'Расположение']
                temp_data = pd.DataFrame(
                    [[car_name, car_rating, car_price, car_location]],
                    columns=cols)

                for item in params_block:
                    col_name = re.findall(r'(.+):', item.text)[0]
                    col_value = re.findall(r':\s(.+)', item.text)[0]
                    temp_data[col_name] = col_value

                for name, value in zip(opt_names, opt_values):
                    temp_data[name] = value

                data = pd.concat([data, temp_data], ignore_index=True)
                logging.info('Успешно получена информация о машине.')
            data.to_csv(
                f'./data_store/parsed_data/data_{self.brand}_{page_num}.csv',
                mode='a',
                index=False,
                encoding='utf-8')
            self.driver.switch_to.window(self.driver.window_handles[0])
            logging.info(f'Успешно получена информация обо всех машинах '
                         f'"{self.brand}" на странице {page_num}.')
        except Exception as err:
            logging.error(
                f'Не удалось получить информацию при парсинге: {err}.',
                exc_info=True)

    def __switch_page(self) -> None:
        ''' 
        Встроенный метод для перелистывания страниц в заданном диапазоне. 
        '''
        for page_num in range(self.page_start, self.page_stop + 1):
            start_time = time.perf_counter()
            self.__get_url(page_num)
            self.__get_links()
            self.__parse_page_to_df(page_num)
            finish_time = time.perf_counter()
            result_time = (finish_time - start_time) / 60
            logging.info(f'Время парсинга бренда "{self.brand}"'
                         f'страницы {page_num}: {result_time} мин.')

    def parse(self, brand: str, page_start: int, page_stop: int) -> None:
        ''' 
        Метод, который будем использовать для парсинга. 
        ---------------------------------------------------
        :brand: str - наименование бренда авто.
        :page_count: int - количество страниц, которое будем парсить.
        '''
        self.brand = brand
        self.page_start = page_start
        self.page_stop = page_stop
        self.links_archive = []
        self.__browser_run()
        self.__switch_page()

In [5]:
parser = AvitoParser(brands_dict)

Перемешаем названия брендов, чтобы в случайном порядке их парсить.

In [9]:
random_brands_list = np.random.permutation(list(brands_dict.keys()))

In [10]:
for brand in tqdm_notebook(random_brands_list):
    parser.parse(brand=brand, page_start=34, page_stop=35)

  0%|          | 0/28 [00:00<?, ?it/s]

На выходе получаем небольшие csv-файлы с данными вида data_bmw_72.csv. В названии указан бренд и номер страницы, с которой в этом csv-файле получены данные о машинах.